In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import chi2_contingency, ttest_ind

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

## Ejercicio 1:
Supongamos que estamos realizando un experimento para determinar si un nuevo diseño de anuncio (test group "ad") tiene un impacto significativo en la tasa de conversión en comparación con el diseño de anuncio anterior (test group "psa").

Objetivo del ejercicio: Comparar las tasas de conversión entre los grupos de prueba "ad" y "psa" para determinar si el nuevo diseño de anuncio es más efectivo en la conversión de usuarios.

Pasos que debemos seguir:

Preparación de los datos: Asegúrate de que los datos estén limpios y listos para el análisis. Esto incluye la eliminación de datos faltantes y la verificación de la coherencia de los datos.

Definición de las hipótesis: Plantea una hipótesis nula (H0) y una hipótesis alternativa (H1) para el experimento (en este caso os las damos definidas):

Hipótesis nula (H0): No hay diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa".

Hipótesis alternativa (H1): Existe una diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa".

Cálculo de la tasa de conversión: Calcula la tasa de conversión para cada grupo:

Para el grupo "ad", la tasa de conversión es el número de usuarios convertidos dividido por el número total de usuarios en ese grupo.

Para el grupo "psa", realiza el mismo cálculo.

Prueba de hipótesis: Utiliza una prueba estadística, como la prueba t de Student o la prueba chi-cuadrado, para comparar las tasas de conversión entre los grupos "ad" y "psa".

Análisis de resultados:

Si el valor p es menor que un umbral predefinido (por ejemplo, 0.05), rechazamos la hipótesis nula y concluimos que hay una diferencia significativa en las tasas de conversión entre los grupos.

Si el valor p es mayor que el umbral, no podemos rechazar la hipótesis nula y concluimos que no hay evidencia suficiente para afirmar que hay una diferencia significativa.

Conclusiones y recomendaciones: Basándote en los resultados de la prueba de hipótesis, llega a una conclusión sobre si el nuevo diseño de anuncio es más efectivo en la conversión de usuarios que el diseño anterior. Si es así, puedes hacer recomendaciones basadas en estos hallazgos.

In [2]:
df = pd.read_csv("marketing_AB.csv", index_col=0)
df.head()

,user id,test group,converted,total ads,most ads day,most ads hour
0,1069124,ad,False,130,Monday,20
1,1119715,ad,False,93,Tuesday,22
2,1144181,ad,False,21,Tuesday,18
3,1435133,ad,False,355,Tuesday,10
4,1015700,ad,False,276,Friday,14


In [3]:
import pandas as pd
import numpy as np
from IPython.display import display
from itertools import combinations
from scipy.stats import kstest, spearmanr, pearsonr


class AutoEDA:
    def __init__(self):
        pass

    def read_file(self, file_path):
        """
        Reads a file and returns a pandas DataFrame.
        
        Parameters:
        - file_path (str): The path to the file.
        
        Returns:
        - DataFrame: A pandas DataFrame.
        """
        # Determine the file extension
        file_extension = file_path.split('.')[-1].lower()
        
        # Read the file based on the file extension
        try:
            if file_extension == 'csv':
                return pd.read_csv(file_path, index_col=0)
            elif file_extension in ['xls', 'xlsx']:
                return pd.read_excel(file_path)
            elif file_extension == 'json':
                return pd.read_json(file_path)
            elif file_extension == 'pkl':
                return pd.read_pickle(file_path)
            else:
                raise ValueError("Reading this format is not yet implemented.")
        except Exception as e:
            return f"File reading failed, error: {e}."


    def explo_df(self, DataFrame, column=None):
        """
        Explores a DataFrame or a specific column and prints various statistics.
        
        Parameters:
        - DataFrame: The DataFrame to explore.
        - column (str or list, optional): The column or columns to explore. If None, explore the entire DataFrame.
        """
        if column is None:
            # General DataFrame exploration
            print("DataFrame Information:")
            display(DataFrame.info())
            print("\nFirst 10 rows of the DataFrame:")
            display(DataFrame.head(10))
            print("\nLast 10 rows of the DataFrame:")
            display(DataFrame.tail(10))
            print("\nStatistical description of the DataFrame (numeric):")
            display(DataFrame.describe().T)
            print("\nStatistical description of the DataFrame (categorical):")
            display(DataFrame.describe(include='object').T)
            print("\nCount of null values per column:")
            display(DataFrame.isnull().sum())
            print("\nPercentage of null values per column (only columns with nulls):")
            null_percentage = round(DataFrame.isnull().sum()/DataFrame.shape[0]*100, 2)
            display(null_percentage[null_percentage > 0])
            print("\nRows with all values as null:")
            all_null_rows = DataFrame[DataFrame.isnull().all(axis=1)]
            if not all_null_rows.empty:
                display(all_null_rows)
            else:
                print("There is no rows with all values as null.")
            print("\nCount of duplicate rows:")
            display(DataFrame.duplicated().sum())
        else:
            # Column(s) exploration
            if isinstance(column, str):
                column = [column]  # Convert to list if a single column is passed as a string
            for col in column:
                print(f"\nExploration of the column: {col}")
                if DataFrame[col].dtype in ['int64', 'float64']:
                    print("\nStatistical description (numeric):")
                    display(DataFrame[col].describe())
                else:
                    print("\nStatistical description (categorical):")
                    display(DataFrame[col].describe(include='object'))
                print("\nCount of null values:")
                display(DataFrame[col].isnull().sum())
                print("\nCount of unique values:")
                display(DataFrame[col].nunique())
                print("\nUnique values:")
                display(DataFrame[col].unique())
                print("\nValue Counts:")
                display(DataFrame[col].value_counts())
                print("\nMost frequent value (mode):")
                display(DataFrame[col].mode().iloc[0])
                print("\nCount of duplicates in the column:")
                display(DataFrame.duplicated(subset=[col]).sum())

In [4]:
df.columns

Index(['user id', 'test group', 'converted', 'total ads', 'most ads day',
       'most ads hour'],
      dtype='object')

In [11]:
autoEda = AutoEDA()
df = autoEda.read_file("marketing_AB.csv")
autoEda.explo_df(df, "most ads hour")


Exploration of the column: most ads hour

Statistical description (numeric):


count    588101.000000
mean         14.469061
std           4.834634
min           0.000000
25%          11.000000
50%          14.000000
75%          18.000000
max          23.000000
Name: most ads hour, dtype: float64


Count of null values:


0


Count of unique values:


...

13    47655
12    47298
11    46210
14    45648
15    44683
10    38939
16    37567
17    34988
18    32323
9     31004
19    30352
21    29976
20    28923
22    26432
23    20166
8     17627
7      6405
0      5536
2      5333
1      4802
3      2679
6      2068
5       765
4       722
Name: most ads hour, dtype: int64


Most frequent value (mode):


13


Count of duplicates in the column:


588077

In [12]:
pd.crosstab(df["test group"], df["converted"], normalize=True)

converted,False,True
test group,,
ad,0.935475,0.024525
psa,0.039286,0.000714


In [32]:
# Tasa de conversión para el grupo "ad"
ad_group = df[df['test group'] == 'ad']
ad_conversion_rate = ad_group['converted'].mean()

# Tasa de conversión para el grupo "psa"
psa_group = df[df['test group'] == 'psa']
psa_conversion_rate = psa_group['converted'].mean()

print(f"Tasa de conversión para el grupo 'ad': {ad_conversion_rate}")
print(f"Tasa de conversión para el grupo 'psa': {psa_conversion_rate}")


Tasa de conversión para el grupo 'ad': 0.025546559636683747
Tasa de conversión para el grupo 'psa': 0.01785410644448223


In [33]:
from scipy.stats import ttest_ind

# Realizar la prueba t de Student
t_stat, p_value = ttest_ind(ad_group['converted'], psa_group['converted'])

print(f"Estadístico t: {t_stat}")
print(f"Valor p: {p_value}")


Estadístico t: 7.37040597428566
Valor p: 1.7033052627831264e-13


In [34]:
# Nivel de significancia
alpha = 0.05

if p_value < alpha:
    print("Rechazamos la hipótesis nula. Hay una diferencia significativa en la tasa de conversión entre los grupos 'ad' y 'psa'.")
else:
    print("No podemos rechazar la hipótesis nula. No hay evidencia suficiente para afirmar que hay una diferencia significativa en la tasa de conversión entre los grupos 'ad' y 'psa'.")


Rechazamos la hipótesis nula. Hay una diferencia significativa en la tasa de conversión entre los grupos 'ad' y 'psa'.
